# 数据加载 / Data Loading

下面先接着上一篇推文的结尾，使用spacy加载英语和德语的`tokenizer`，以及加载`torchtext`的`multi30K`数据集。

## spaCy
首先使用**spaCy**库加载`tokenizer`模型，执行下面代码即可自动下载。不过需要注意的是，需要科学上网。在安装**spaCy**时，需要注意版本`spacy==3.2`  `pydantic==1.10.12`，否则在后续代码中可能会出错。

In [ ]:
# 如果是在notebook中运行，需要使用下面的命令设置代理
%env ALL_PROXY=http://172.29.48.1:7890
%env HTTP_PROXY=http://172.29.48.1:7890
%env HTTPS_PROXY=http://172.29.48.1:7890

In [ ]:
import spacy, os
def load_tokenizers():
    try:
        spacy_de = spacy.load("de_core_news_sm")
    except IOError:
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")
    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")
    return spacy_de, spacy_en

def tokenize(text, tokenizer): # tokenizer是spacy的tokenizer
    return [tok.text for tok in tokenizer.tokenizer(text)] # 返回一个list

def yield_tokens(data_iter, tokenizer, index): # data_iter是一个迭代器，tokenizer是spacy的tokenizer，index是from或者to
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])

## Multi30k 德译英数据集
下面是下载`Multi30`中德语和英语语料的数据集，由于`torchtext`中的`multi30k`数据集默认下载地址已经停止维护了，所以需要手动改成下面的下载地址。

In [ ]:
from torchtext.datasets import multi30k
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"
multi30k.URL["test"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz"

multi30k.MD5["train"] = "20140d013d05dd9a72dfde46478663ba05737ce983f478f960c1123c6671be5e"
multi30k.MD5["valid"] = "a7aa20e9ebd5ba5adce7909498b94410996040857154dab029851af3a866da8c"
multi30k.MD5["test"] = "6d1ca1dba99e2c5dd54cae1226ff11c2551e6ce63527ebb072a1f70f72a5cd36"

## 构建词汇表
`Torchtext` 是 `PyTorch` 的一个子库，专门用于处理文本数据。在自然语言处理（NLP）任务中，词典（Vocab）是一个基础且重要的组件，它负责将词元（`tokens`）映射到索引（`indices`），从而使文本数据能够被模型处理。`Torchtext` 提供了一个名为 `Vocab` 的类，用于创建和使用词典。

在 `Torchtext` 中，可以通过两种方式创建词典：

1. 使用有序字典（`OrderedDict`）：首先，通过统计词频来创建一个有序字典，然后将其传递给` vocab` 函数来创建词典对象。这种方法保证了词典中词元的顺序反映了它们在有序字典中的插入顺序。

2. 使用可迭代对象：可以通过 `build_vocab_from_iterator` 函数从一个迭代器中创建词典。这个迭代器必须产生词元列表或其他可迭代的词元。

@Caution \
在使用`build_vocab_from_iterator`时，需要注意以下几点：
1. 迭代器的设计：迭代器应该能够有效地遍历数据集中的所有文本，并将其分解为单词。
2. 频率阈值：`min_freq`参数允许你指定一个单词必须出现的最小次数，才能被包含在词汇表中。这有助于去除罕见的单词，减少噪声。
3. 特殊符号：`specials`参数允许你添加一些特殊的符号，如未知词标记<unk>或填充符<pad>。这些符号在训练模型时非常有用。
4. 词汇表的顺序：`special_first`参数决定了特殊符号是添加到词汇表的开始还是结束。这可能会影响模型的训练和预测

In [ ]:

import torchtext.datasets as datasets
from torchtext.vocab import build_vocab_from_iterator
import os
import torch
    
def build_vocabulary(spacy_de, spacy_en):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    print("Building German Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    for t in test:
        print(t)
    vocab_src = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_de, index=0), # 从train, val, test中提取出来的是一个tuple，所以index=0表示提取德语语句
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )
    print(vocab_src)
    print("Building English Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_tgt = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_en, index=1), # 从train, val, test中提取出来的是一个tuple，所以index=1表示提取英语语句
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_tgt.set_default_index(vocab_tgt["<unk>"])

    return vocab_src, vocab_tgt


def load_vocab(spacy_de, spacy_en):
    if not os.path.exists("vocab.pt"):
        vocab_src, vocab_tgt = build_vocabulary(spacy_de, spacy_en)
        torch.save((vocab_src, vocab_tgt), "vocab.pt")
    else:
        vocab_src, vocab_tgt = torch.load("vocab.pt")
    print("Finished.\nVocabulary sizes:")
    print(len(vocab_src))
    print(len(vocab_tgt))
    return vocab_src, vocab_tgt


# if is_interactive_notebook():
    # global variables used later in the script
spacy_de, spacy_en = load_tokenizers()
vocab_src, vocab_tgt = load_vocab(spacy_de, spacy_en) # show_example(load_vocab, args=[spacy_de, spacy_en])

到此，我们得到了德语的tokenizer，英语的tokenizer，源语言德语的词表和目标语言英语的词表。

# DataLoader / torch.utils.data.DataLoader
接下来重点要讲的是Pytorch中的`Dataloader`。

PyTorch中的`torch.utils.data.DataLoader`类是一个功能强大的有效加载数据的工具。它在数据集上提供了一个可迭代对象，支持地图样式和*可迭代样式*的数据集，并提供了**自定义数据加载顺序**、**自动批处理**、**单进程**和**多进程数据加载**以及**自动内存固定等功能**。

@Dot_List
@Title Parameters
- **dataset** (`Dataset`) – 加载数据的来源数据集。
- **batch_size** (`int`, 可选) – 每批加载的样本数量（默认值：`1`）。
- **shuffle** (`bool`, 可选) – 设置为 True 以在每个训练周期打乱数据（默认值：`False`）。
- **sampler** (`Sampler` 或 `Iterable`, 可选) – 定义从数据集中抽取样本的策略。可以是任何实现了 `__len__` 的 `Iterable`。如果指定了此参数，则不能指定 `shuffle`。
- **batch_sampler** (`Sampler` 或 `Iterable`, 可选) – 类似于采样器，但一次返回一批索引。与`batch_size`、`shuffle`、`sampler`和`drop_last`互斥。
- **num_workers** (`int`, 可选) – 用于数据加载的子进程数量。`0` 表示数据将在主进程中加载。（默认值：`0`）
- **collate_fn** (`Callable`, 可选) – 将样本列表合并成一个小批量 `Tensor`。用于从映射样式数据集中批量加载时。
- **pin_memory** (`bool`, 可选) – 如果为 `True`，数据加载器将在返回数据之前将 `Tensors` 复制到`device/CUDA`的固定内存中。如果您的数据元素是自定义类型，或者您的合并函数返回一个自定义类型的批量，请参见下面的示例。
- **drop_last** (`bool`, 可选) – 设置为 `True` 以丢弃最后一批不完整的样本，如果数据集大小不能被批量大小整除。如果为 `False` 而数据集大小不能被批量大小整除，则最后一批将较小。（默认值：`False`）
- **timeout** (`numeric`, 可选) – 如果为正，则为从工作进程收集一批的超时值。应始终为非负值。（默认值：`0`）
- **worker_init_fn** (`Callable`, 可选) – 如果不为 `None`，此函数将在每个工作子进程上调用，输入为工作进程 `ID`（一个在 [0, num_workers - 1] 范围内的整数），在设定随机种子后、数据加载之前调用。（默认值：None）
- **multiprocessing_context** (`str` 或 `multiprocessing.context.BaseContext`, 可选) – 如果为 None，将使用操作系统的默认多进程上下文。（默认值：None）
- **generator** (`torch.Generator`, 可选) – 如果不为 `None`，此随机数生成器将由 `RandomSampler` 用于生成随机索引，并用于多进程生成工作进程的基础种子。（默认值：`None`）
- **prefetch_factor** (`int`, 可选，仅限关键字参数) – 每个工作进程提前加载的批次数。`2`表示所有工作进程总共将预取 `2 * num_workers`个批次。（默认值取决于 `num_workers` 的设置值。如果 `num_workers=0`，则默认值为 `None`。否则，如果 `num_workers > 0`，默认值为 `2`。）
- **persistent_workers** (`bool`, 可选) – 如果为 `True`，数据加载器在数据集被消耗一次后将不关闭工作进程。这允许保持工作进程的数据集实例存活。（默认值：`False`）
- **pin_memory_device** (`str`, 可选) – 如果固定内存为 `True`，则固定内存的设备。

## Map-style Dataset and Iterable-style Dataset
`DataLoader`构造函数最重要的参数是数据集，它可以是`Map-style`，也可以是`Iterable-style`。
1. `Map-style`是Pytorch中默认的数据集加载方式，它表示一个从`indices/keys`到数据样本的映射表，可以通过**索引**或**键**来访问数据样本，例如，有一个数据集`dataset`，包含有一张图片和对应的`label`，我们要访问`idx-th`条数据时，就可以通过`dataset[idx]`访问到，`Map-style datasets`的实现需要继承`torch.utils.data.Dataset`。


@Cite_1 \
所有表示从`indices/keys`到数据样本映射的数据集都应该继承它。必须重写`__getitem__()`，以支持获取给定键的数据样本。子类也可以选择性地重写__len__()，许多`Sampler`实现和`DataLoader`的默认选项预计将返回数据集的大小。子类也可以选择性地实现`__getitems__()`，以加速批处理样例加载。该方法接受批次样本的指标列表，并返回样本列表。

下面这个是两个简单示例。

In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        # 返回索引对应的样本
        return self.data[index]

    def __len__(self):
        # 返回数据集的长度
        return len(self.data)

# 创建数据集对象
data_list = [11, 22, 33, 44, 55, 66, 77, 88, 99]
dataset = CustomDataset(data_list)

# 创建数据集加载器
data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)
for step, batch_x in enumerate(data_loader):
    print(f' Step: {step} | batch x: {batch_x.numpy()} ')


In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

下面是`Dataloader`在获取一个batch数据时的源码，主要是通过`_MapDatasetFetcher`类来实现，可以看到在一次生成一个`batch_size`大小的数据时，返回的是一个数据样本列表，否则返回的是一个数据样本。

In [ ]:
class _MapDatasetFetcher(_BaseDatasetFetcher):
    def __init__(self, dataset, auto_collation, collate_fn, drop_last):
        super(_MapDatasetFetcher, self).__init__(dataset, auto_collation, collate_fn, drop_last)

    def fetch(self, possibly_batched_index):
        if self.auto_collation:
            data = [self.dataset[idx] for idx in possibly_batched_index]
        else:
            data = self.dataset[possibly_batched_index]
        return self.collate_fn(data)



2. `Iterable-style`是Pytorch中的另一种数据集加载方式。这种方式更加灵活，可以在样本读取过程中进行一些数据处理操作。Iterable-Style数据集加载方式适用于那些数据不能一次性加载到内存中的情况，比如大型数据集或需要实时生成的数据集。
通过继承`torch.utils.data.IterableDataset`实现，需要重写`__iter__()`方法

@Cite_2 \
在使用`Dataloader`中使用`IterableDataset`时，数据集中的数据是通过迭代器产生，如下面给出的代码示例。当`num_workers>0`时，每个工作进程`worker`将拥有数据集对象的不同拷贝副本，因此通常需要保证每个进程获取到的数据不重合，以避免从工作进程返回重复的数据。当在工作进程中调用`torch.utils.data.get_worker_info()`时，返回有关该工作进程的信息，可以获取到该进程的`worker_id`，据此来保证每个进程的数据集分片不重叠。它可以在数据集的`__iter__()`方法中使用，也可以在`DataLoader`的`worker_init_fn`选项中使用，以修改每个副本的行为。

下面这个代码演示自定义一个`IterableDataset`类，分单进程和多进程实现`Iterable`。单线程就是返回一个数据集的迭代器，当进程为2时，则是将数据集分成两等分，每个进程一个迭代器返回数据。

In [ ]:
import torch
import math
from torch.utils.data import DataLoader, IterableDataset
class MyIterableDataset(IterableDataset):
    def __init__(self, start, end):
        super(MyIterableDataset).__init__()
        assert end > start, "this example code only works with end >= start"
        self.start = start
        self.end = end
    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None:  # single-process data loading, return the full iterator
            iter_start = self.start
            iter_end = self.end
            return iter(range(iter_start, iter_end))
        else:  # in a worker process
            # split workload
            per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
            worker_id = worker_info.id
            iter_start = self.start + worker_id * per_worker
            iter_end = min(iter_start + per_worker, self.end)
            print(f"Woker {worker_id} fetched [{iter_start}, {iter_end}].")
            return iter(range(iter_start, iter_end))
ds = MyIterableDataset(start=3, end=20)

data_loader=DataLoader(ds, batch_size=4, num_workers=0)
for step, batch_x in enumerate(data_loader):
    print(f' Step: {step} | batch x: {batch_x.numpy()} ')
print("#"*50)
data_loader=DataLoader(ds, batch_size=4, num_workers=2)
for step, batch_x in enumerate(data_loader):
    print(f' Step: {step} | batch x: {batch_x.numpy()} ')
print("#"*50)
data_loader=DataLoader(ds, batch_size=4, num_workers=12)
for step, batch_x in enumerate(data_loader):
    print(f' Step: {step} | batch x: {batch_x.numpy()} ')
print("#"*50)

<p align="cneter" width="80%">
    <img src="./imgs/dataloader/iter_dataloader_mp.png">
</p>
从这个示例中，就可以看出当设置为多进程时，每个进程获取到的数据是不重叠的，特别需要注意的是，每个进程获取到的数据量是`math.ceil(len(datasets)/worker_num)`，是向上取整的，这样保证了当数据量少于进程数时，也能够返回数据集。从图中标红部分还可以看出，对于多余进程获取的数据集实际上是空集。巧妙的避免了很多边界情况。

下面是通过`worker_init_fn`来实现多进程去重的示例。

In [ ]:
import math
class MyIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, start, end):
        super(MyIterableDataset).__init__()
        assert end > start, "this example code only works with end >= start"
        self.start = start
        self.end = end
    def __iter__(self):
        return iter(range(self.start, self.end))
# should give same set of data as range(3, 7), i.e., [3, 4, 5, 6].
ds = MyIterableDataset(start=3, end=7)

# Single-process loading
print(list(torch.utils.data.DataLoader(ds, num_workers=0)))
# Directly doing multi-process loading yields duplicate data
print(list(torch.utils.data.DataLoader(ds, num_workers=2)))

# Define a `worker_init_fn` that configures each dataset copy differently
def worker_init_fn(worker_id):
    worker_info = torch.utils.data.get_worker_info()
    dataset = worker_info.dataset  # the dataset copy in this worker process
    overall_start = dataset.start
    overall_end = dataset.end
    # configure the dataset to only process the split workload
    per_worker = int(math.ceil((overall_end - overall_start) / float(worker_info.num_workers)))
    worker_id = worker_info.id
    dataset.start = overall_start + worker_id * per_worker
    dataset.end = min(dataset.start + per_worker, overall_end)

# Mult-process loading with the custom `worker_init_fn`
# Worker 0 fetched [3, 4].  Worker 1 fetched [5, 6].
print(list(torch.utils.data.DataLoader(ds, num_workers=2, worker_init_fn=worker_init_fn)))

# With even more workers
print(list(torch.utils.data.DataLoader(ds, num_workers=12, worker_init_fn=worker_init_fn)))

## 数据加载顺序和采样器
1. 对于`Iterable-style Dataset`来说，数据的加载顺序完全取决于用户定于的迭代器，这使得按照chunk读取以及使用动态批大小变得更加容易。
2. 对于`map-style`的数据集，` torch.utils.data.Sampler`被用来指定在加载数据过程中`indices/keys`的顺序。当设置`Dataloader`参数`shuffle=True`时，会自动构造打乱的采样器，否则按顺序采样。也可以通过指定`sampler`参数来自定义采样器方法。`PyTorch`提供了几个内置采样器，如`SequentialSampler`， `RandomSampler`和`SubsetRandomSampler`。自定义采样器也可以通过子类化`torch.utils.data.Sampler`来创建

下面看一个自定义`Sampler`的示例：

In [ ]:
from typing import Iterator, List
from torch.utils.data import Sampler
class AccedingSequenceLengthSampler(Sampler[int]):
    def __init__(self, data: List[str]) -> None:
        self.data = data
    def __len__(self) -> int:
        return len(self.data)
    def __iter__(self) -> Iterator[int]:
        sizes = torch.tensor([len(x) for x in self.data])
        yield from torch.argsort(sizes).tolist()
class AccedingSequenceLengthBatchSampler(Sampler[List[int]]):
    def __init__(self, data: List[str], batch_size: int) -> None:
        self.data = data
        self.batch_size = batch_size
    def __len__(self) -> int:
        return (len(self.data) + self.batch_size - 1) // self.batch_size
    def __iter__(self) -> Iterator[List[int]]:
        sizes = torch.tensor([len(x) for x in self.data])
        for batch in torch.chunk(torch.argsort(sizes), len(self)):
            yield batch.tolist()

@Caution \
1. 这里的`torch.argsort`返回的是排序后，每个元素在排序前列表中的下标。
2. 在使用`torch.nn.parallel.DistributedDataParallel`也就是**分布式数据并行（DDP）**时，需要使用`torch.utils.data.distributed.DistributedSampler`，用来加载原始数据集的一个子集。

## Automatic Batching
`DataLoader`通过`batch_size`、`drop_last`、`batch_sampler`和`collate_fn`参数支持自动批处理。当指定`batch_size`时，`Dataloader`生成成批样本，而不是单个样本。`batch_size`和`drop_last`参数用于指定数据加载器如何获取批量的**batches of dataset keys**。对于`Map-style`的数据集，用户可以自定义`batch_sampler`，它一次生成一个`key`列表。

@Caution
1. batch_size和drop_last参数本质上用于从sampler构造batch_sampler。对于地图样式的数据集，采样器要么由用户提供，要么根据shuffle参数构造。对于可迭代风格的数据集，采样器是一个虚拟的无限采样器。
2. 当使用多进程处理从可迭代风格的数据集获取时，`drop_last`参数会删除每个`worker`的数据集副本的**最后一批非完整批**。

## collate_fn的作用
`collate_fn`参数可用于**自定义如何将单个样本整理成batch**，`collate_fn`函数是作用域`Sampler`之后，也就是获得了一个批次的数据样本列表后，再传给`collate_fn`进行处理。

当`batch_size`和`batch_sampler`都为`None`时，`Dataloader`将禁用**Automatic Batching**，`collate_fn`则只是简单的将`Numpy array`转成`Pytorch tensors`，其他内容不做任何改变。

当`Dataloader`启用**Automatic Batching**，`collate_fn`会将传入的`a list of data samples`转换成用于`dataloader iterator`的`batch`数据。默认的`collate_fn`的作用主要有以下三点：
1. 它总是添加一个新维度作为批处理维度。 
2. 它会自动将`NumPy array`和Python数值转换为`PyTorch tansor`。 
3. 它保留了数据结构，例如，如果每个样本都是一个字典，它输出一个字典，具有相同的`key`，但`value`是`batched Tensors`。

用户可以自定义collate_fn函数，自己来决定如何进行分批。


## Single-and Multi-process Data Loading
默认情况下，`DataLoader`使用单进程数据加载。但是，可以通过将`num_workers`参数设置为正整数来启用**多进程数据加载**。这允许跨多个子进程并行加载数据，从而提高大型数据集的性能

## Memory Pinning
为了更快地将数据传输到支持`cuda`的`gpu`，可以将`pin_memory`参数设置为`True`。这确保了获取的数据张量被放置在固定内存中，从而允许更快的主机到gpu的复制过程。

@Caution
如果设置了自定义的`collate_fn`或者每个batch中的元素是自定义类型，则默认的`Memory Pinning`不执行，直接返回元素，用户需要自定义一个`pin_memory()`方法来处理。如下面这个代码示例。

In [1]:
from torch.utils.data import TensorDataset, DataLoader
import torch
class SimpleCustomBatch:
    def __init__(self, data):
        transposed_data = list(zip(*data))
        self.inp = torch.stack(transposed_data[0], 0)
        self.tgt = torch.stack(transposed_data[1], 0)

    # custom memory pinning method on custom type
    def pin_memory(self):
        self.inp = self.inp.pin_memory()
        self.tgt = self.tgt.pin_memory()
        return self

def collate_wrapper(batch):
    return SimpleCustomBatch(batch)

inps = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
tgts = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
dataset = TensorDataset(inps, tgts)

loader = DataLoader(dataset, batch_size=2, collate_fn=collate_wrapper,
                    pin_memory=True)

for batch_ndx, sample in enumerate(loader):
    print(sample.inp.is_pinned())
    print(sample.tgt.is_pinned())

/opt/conda/envs/transformer/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
True
True
True
True
True
True
True
True
True


# 回到正题 / Call back
下面，接着讲手写Transformer过程中，训练数据是如何加载的。首先，我们自定义一个`collate_fn`函数，来实现对每个`batch`中的数据进行`padding`处理，保证每个`batch`中数据的长度均匀，便于后续的训练。`padding`的方式主要是先分别在数据前后加上`<s>`和`</s>`token，然后再在末尾添加`padding_id`使数据长度一致。由此，下面这段便很容易看懂。

In [ ]:
from torch.nn.functional import log_softmax, pad
def collate_batch(
    batch,
    src_pipeline,
    tgt_pipeline,
    src_vocab,
    tgt_vocab,
    device,
    max_padding=128,
    pad_id=2,
):
    bs_id = torch.tensor([0], device=device)  # <s> token id
    eos_id = torch.tensor([1], device=device)  # </s> token id
    src_list, tgt_list = [], []
    for (_src, _tgt) in batch:
        processed_src = torch.cat( # 在0维度上拼接<s>和</s>
            [
                bs_id,
                torch.tensor(
                    src_vocab(src_pipeline(_src)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    tgt_vocab(tgt_pipeline(_tgt)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        src_list.append( # 在0维度上拼接padding
            # warning - overwrites values for negative values of padding - len
            pad(
                processed_src,
                (
                    0,
                    max_padding - len(processed_src),
                ),
                value=pad_id,
            )
        )
        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_padding - len(processed_tgt)),
                value=pad_id,
            )
        )

    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)
    return (src, tgt)

下面这段代码则是非常简单的创建`Dataloader`的代码，有了前面的基础之后，已经可以很轻易的看懂。不在赘述细节。

In [ ]:
from torch.utils.data.distributed import DistributedSampler
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
def create_dataloaders(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    batch_size=12000,
    max_padding=128,
    is_distributed=True,
):
    # def create_dataloaders(batch_size=12000):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"],
        )

    train_iter, valid_iter, test_iter = datasets.Multi30k(
        language_pair=("de", "en")
    )

    train_iter_map = to_map_style_dataset(
        train_iter
    )  # DistributedSampler needs a dataset len()
    train_sampler = (
        DistributedSampler(train_iter_map) if is_distributed else None
    )
    valid_iter_map = to_map_style_dataset(valid_iter)
    valid_sampler = (
        DistributedSampler(valid_iter_map) if is_distributed else None
    )

    train_dataloader = DataLoader(
        train_iter_map,
        batch_size=batch_size,
        shuffle=(train_sampler is None),
        sampler=train_sampler,
        collate_fn=collate_fn,
    )
    valid_dataloader = DataLoader(
        valid_iter_map,
        batch_size=batch_size,
        shuffle=(valid_sampler is None),
        sampler=valid_sampler,
        collate_fn=collate_fn,
    )
    return train_dataloader, valid_dataloader

In [ ]:
train_dataloader, valid_dataloader = create_dataloaders(
        gpu=0,
        vocab_src=vocab_src,
        vocab_tgt=vocab_tgt,
        spacy_de=spacy_de,
        spacy_en=spacy_en,
        batch_size=32, # config["batch_size"] // ngpus_per_node,
        max_padding=72, #config["max_padding"],
        is_distributed=False, # config["distributed"],
    )

到此，`Dataloader`部分结束，想更加深入了解的，建议直接看`Pytorch`官方文档，写得算是比较清楚了，同时，也强烈建议看看源码，源码也是比较容易看懂的。

下一篇推文将详细讲讲训练的一些细节。